In [ ]:
import os
import shutil
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import pydicom
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle as pkl
def dcm2nii(path_read, path_save):
    '''
    ## Convert Dicom Series Files to a Single NII File
    ### Args:
        path_read: The file folder containing dicom series files(No other files exits)
        path_save: The path you save the .nii/.nii.gz data file
    '''
    # GetGDCMSeriesIDs读取序列号相同的dcm文件
    series_id = sitk.ImageSeriesReader.GetGDCMSeriesIDs(path_read)
    # GetGDCMSeriesFileNames读取序列号相同dcm文件的路径，series[0]代表第一个序列号对应的文件
    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(path_read, series_id[0])
    series_reader = sitk.ImageSeriesReader()
    series_reader.SetFileNames(series_file_names)
    image3d = series_reader.Execute()
    sitk.WriteImage(image3d, path_save)

In [ ]:
def position(dicom_dir):
    # Get single dcm's physical position
    file = pydicom.filereader.dcmread(dicom_dir)
    cosines = file[0x20, 0x37].value
    ipp = file[0x20, 0x32].value
    a1 = np.zeros((3,))
    a2 = np.zeros((3,))
    for i in range(3):
        a1[i] = cosines[i]
        a2[i] = cosines[i+3]
    index = np.abs(np.cross(a1, a2))
    return ipp[np.argmax(index)]

In [ ]:
def Rec_crop2D(mask,index):
    x_l,x_r,y_d,y_u=0,0,0,0
    for i in range(len(mask[index])):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_u=i-1
            break
    for i in range(len(mask[index])-1,0,-1):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_d=i+1
            break
    mask_t=np.transpose(mask[index])
    for i in range(len(mask_t)-1,0,-1):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_r=i+1
            break
    for i in range(len(mask_t)):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_l=i-1
            break
    # print(x_l,x_r,y_u,y_d)
    croped_mask=mask[index][y_u:y_d,x_l:x_r]
    return croped_mask,[x_l,x_r,y_u,y_d]
def crop(mask):
    x_l,x_r,y_d,y_u=0,0,0,0
    for i in range(len(mask)):
        if np.count_nonzero(mask[i]>0)>0:
            index=i
    for i in range(len(mask[index])):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_u=i-1
            break
    for i in range(len(mask[index])-1,0,-1):
        if np.count_nonzero(mask[index][i]==1)>0:
            y_d=i+1
            break
    mask_t=np.transpose(mask[index])
    for i in range(len(mask_t)-1,0,-1):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_r=i+1
            break
    for i in range(len(mask_t)):
        if np.count_nonzero(mask_t[i]==1)>0:
            x_l=i-1
            break
    # print(x_l,x_r,y_u,y_d)
    # croped_mask=mask[index][y_u:y_d,x_l:x_r]
    return x_l,x_r,y_u,y_d



In [ ]:
def resample_dataset2(spacing=[0.994,1.826],datapath='E:/VST_fusion/dataset'):
#Map resolution from nifti to nifti
    error=[]
    for space in spacing:
        for roots,dirs,files in os.walk(datapath):
            if len(dirs)==0:
                files=exclude_seg_files(os.listdir(roots))
                for i in range(len(files)):
                    data_path=os.path.join(roots,files[i])
                    pre_data=sitk.ReadImage(data_path)
                    data_spacing=list(pre_data.GetSpacing())
                    data_spacing[0]=space
                    data_spacing[1]=space
                    new_data=resample_volume(Origin=data_path,new_spacing=data_spacing,output='')
                    if len(sitk.GetArrayFromImage(new_data))!=25 and 'LGE' not in data_path:
                        print(f'frame resample error: {data_path} {sitk.GetArrayFromImage(new_data).shape}')
                        error.append(data_path)
                    else:
                        save_path=roots.replace('nifti',f'nifti_spacing_{space}')
                        save_path=save_path.replace('dataset',f'dataset_spacing_{space}')
                        # save_path=save_path.replace('E:','F:')
                        if not os.path.exists(save_path):
                            os.makedirs(save_path)
                        save_path=os.path.join(save_path,files[i])
                        sitk.WriteImage(new_data,save_path)
                        print(f'{save_path} saved')
    return error
error=resample_dataset2(datapath='/Users/airskcer/Downloads/NP_nifti-20230109/',spacing=[0.994,1.826])
# error=resample_dataset2(datapath='/Users/airskcer/Downloads/RCM_nifti-20230107/',spacing=[0.994,1.826])

In [ ]:
def check_data(root='/Users/airskcer/Downloads/HCM_nifti-20230102/',mod='4CH_data',output='/Users/airskcer/Downloads/check'):
#save nifti as jpeg to check data
    if not os.path.exists(output):
        os.makedirs(output)
    for roots,dirs,files in os.walk(root):
        if len(dirs)==0 and mod in roots:
            files=exclude_seg_files(files)
            # if len(files)<2:
            #     continue
            for i in range(len(files)):
                data=sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(roots,files[i])))
                cv2.imwrite(os.path.join(output,files[i].replace('.nii.gz','.jpg')),norm_img(data[0]))
# check_data()

In [ ]:
def save_cine(dataroot='./New_dicom/',output='./',fps=25,thres=60):
    """Convert dicom data files into nifti format, and resample its resolution
    Args:
        dataroot (str, optional): The root path of resampled data. Defaults to './New_dicom/'.
        output (str, optional): The output path of generated nfiti data. Defaults to './'.
        fps (int, optional): The fps of a single slice(SAX/LAX4CH). Defaults to 25.
    """
    error=[]
    for roots,dirs,files in os.walk(dataroot):
        if len(dirs)==0:
            try:
                slice_map={}
                file_info={}
                files=exclude_seg_files(files)
                for i in range(len(files)):
                    info=pydicom.dcmread(os.path.join(roots,files[i]))
                    file_info[files[i]]=info
                    sn=info.SeriesNumber
                    if sn not in list(slice_map.keys()):
                        slice_map[sn]=[files[i]]
                    else:
                        slice_map[sn].append(files[i])
                slices=list(slice_map.keys())
                for i in range(len(slices)):
                    dcms=slice_map[slices[i]]
                    pats_name=str(file_info[dcms[0]]['0010','0010'].value).replace(' ','_')
                    pats_id=file_info[dcms[0]]['0010','0020'].value
                    filetag=str(file_info[dcms[0]]['0008','103e'].value)
                    print(filetag)
                    # if ('BH' in filetag or '4CH' in filetag or '4 CH' in filetag or 'TFE' in filetag or 'SecondaryCapture' in filetag or '1'==filetag) and ('SA' not in filetag and 'sa' not in filetag and 'shot' not in filetag and '8slices' not in filetag and 'LVOT' not in filetag):
                    if len(files)<thres and 'LVOT' not in filetag:
                        print(f'fps: {len(dcms)}')
                        # if len(dcms)%fps==0:
                        if len(dcms)>=10:
                            temp_folder=os.path.join(output,'4ch_temp')
                            savepath=os.path.join(output,'4CH_data',f'{pats_id}_{pats_name}')
                            filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            # continue
                        else:
                            # temp_folder=os.path.join(output,'4ch_lge_temp')
                            # savepath=os.path.join(output,'4CH_LGE_data',f'{pats_id}_{pats_name}')
                            # filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                        # if True:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass
                        shutil.rmtree(temp_folder)
                    # elif 'SA' in filetag or 'sa' in filetag or '2'==filetag or 'shot' in filetag or 'PSIR_TFE 8slices'==filetag:
                    elif len(files)>=thres:
                        print(f'fps: {len(dcms)}')
                        """
                        Bigger slice ID, lower z-axis position
                        """
                        # if len(dcms)%fps==0:
                        # if len(dcms)>=25:
                        if True:
                            temp_folder=os.path.join(output,'sax_temp')
                            savepath=os.path.join(output,'SAX_data',f'{pats_id}_{pats_name}')
                            filename=f'slice_{slices[i]}.nii.gz'
                            # continue
                        else:
                            # temp_folder=os.path.join(output,'sax_lge_temp')
                            # savepath=os.path.join(output,'SAX_LGE_data',f'{pats_id}_{pats_name}')
                            # filename=f'{pats_id}_{pats_name}_{slices[i]}.nii.gz'
                            continue
                        for j in range(len(dcms)):
                            if not os.path.exists(temp_folder):
                                os.makedirs(temp_folder)
                            shutil.copyfile(os.path.join(roots,dcms[j]),os.path.join(temp_folder,dcms[j]))
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)
                        try:
                            dcm2nii(temp_folder,os.path.join(savepath,filename))
                            # resample_volume(Origin=os.path.join(savepath,filename),output=os.path.join(savepath,filename))
                            print(f'{filename} saved into {savepath}')
                        except:
                            pass 
                        shutil.rmtree(temp_folder)
            except:
                print(f'unknow error occured with {roots}')
                error.append(roots)
    temps=['4ch_temp','4ch_lge_temp','sax_temp','sax_lge_temp']
    for i in range(len(temps)):
        try:
            shutil.rmtree(os.path.join(output,temps[i]))
        except:
            pass
    return error
cine_error=save_cine(dataroot='/Users/airskcer/Downloads/HCM_Resampled_dicom-20230112/',output='/Users/airskcer/Downloads/HCM_nifti-20230112/',fps=25,thres=80)
# save_cine(dataroot='/Users/airskcer/Downloads/RCM_Resampled_dicom-20230107/11124028_SHI_KE_GANG/',output='/Users/airskcer/Downloads/RCM_nifti-20230107/',fps=25,thres=60)